<a href="https://colab.research.google.com/github/IA2021UR/entregable-3-joferne/blob/main/entregable3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entregable 3

Como no consigo subir a GitHub el archivo porque pesa demasiado, dejo aquí el enlace.

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

Consiste en crear un detector de noticias verdaderas y falsas.

Cargamos los datos y preparamos el dataset.

In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df_true = pd.read_csv('True.csv')
df_fake = pd.read_csv('Fake.csv')

df_train_true, df_test_true = train_test_split(df_true, test_size=0.2)
df_train_fake, df_test_fake = train_test_split(df_fake, test_size=0.2)

Creamos las carpetas.

In [3]:
import os

os.mkdir("dataSet")
os.mkdir("dataSet/train")
os.mkdir("dataSet/train/fake")
os.mkdir("dataSet/train/true")
os.mkdir("dataSet/test")
os.mkdir("dataSet/test/fake")
os.mkdir("dataSet/test/true")

In [4]:
df_train_true.to_csv('dataSet/train/true/true_train.csv')
df_test_true.to_csv('dataSet/test/true/true_Test.csv')

df_train_fake.to_csv('dataSet/train/fake/fake_train.csv')
df_test_fake.to_csv('dataSet/test/fake/fake_test.csv')

Descargamos la librería fastai.

In [5]:
!pip install fastai --upgrade

     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


Tokenización con fastai.

In [6]:
from fastai.text.all import *
import pathlib

path = pathlib.Path('dataSet/')

In [7]:
files = get_files(path, folders=['test', 'train'])
txt = files[2].open().read(); txt[:75]

',title,text,subject,date\n6502, Whoops: VA Gun Shop Employee Accidentally Sh'

In [8]:
wt = WordTokenizer()
tkn = Tokenizer(wt)

In [9]:
first(wt([txt]))

(#9749410) [',','title',',','text',',','subject',',','date','\n','6502'...]

In [10]:
coll_repr(tkn(txt), 31)

'(#10907072) [\'xxbos\',\',\',\'title\',\',\',\'text\',\',\',\'subject\',\',\',\'date\',\'\\n\',\'6502\',\',\',\'xxmaj\',\'whoops\',\':\',\'xxup\',\'va\',\'xxmaj\',\'gun\',\'xxmaj\',\'shop\',\'xxmaj\',\'employee\',\'xxmaj\',\'accidentally\',\'xxmaj\',\'shoots\',\'herself,"an\',\'employee\',\'of\',\'xxmaj\'...]'

Numericalización con fastai.

In [11]:
txts = L(o.open().read() for o in files[:2000])
toks200 = txts[:200].map(tkn)
num = Numericalize()
num.setup(toks200)

In [12]:
coll_repr(num.vocab,20)

"(#60008) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','to','of','a','and','in','that','on','-'...]"

In [13]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))
nums = num(toks)[:20]
nums

(#10907072) ['xxbos',',','title',',','text',',','subject',',','date','\n','6502',',','xxmaj','whoops',':','xxup','va','xxmaj','gun','xxmaj','shop','xxmaj','employee','xxmaj','accidentally','xxmaj','shoots','herself,"an','employee','of','xxmaj'...]


TensorText([    2,    10,  3509,    10,  2935,    10,  1496,    10,  1877,    61,
            0,    10,     8, 32046,    38,     7,  3959,     8,   442,     8])

In [14]:
' '.join(num.vocab[o] for o in nums)

'xxbos , title , text , subject , date \n xxunk , xxmaj whoops : xxup va xxmaj gun xxmaj'

Organizamos el dataset en batches.

In [15]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)

In [16]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [17]:
print(' '.join(num.vocab[o] for o in x[0][:20]))
print(' '.join(num.vocab[o] for o in y[0][:20]))

xxbos , title , text , subject , date 
 xxunk xxup redux ? cnn ’s xxmaj role in xxmaj
, title , text , subject , date 
 xxunk xxup redux ? cnn ’s xxmaj role in xxmaj peddling


Modelo de lenguaje.

In [18]:
df_fake = pd.read_csv('Fake.csv')
df_fake['class'] = 'fake'

df_true = pd.read_csv('True.csv')
df_true['class'] = 'true'

dfFastAI = pd.concat([df_true, df_fake])

os.mkdir("dataSetConcatenated")
dfFastAI.to_csv('dataSetConcatenated/concatenated.csv')

Lo hacemos con la documentación de:

https://docs.fast.ai/text.data.html#TextDataLoaders.from_csv

In [19]:
df = pd.read_csv('dataSetConcatenated/concatenated.csv')

dls_lm = TextDataLoaders.from_csv('dataSetConcatenated/', 'concatenated.csv', text_col='text', is_lm=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [20]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj here we go again 25 years after the xxmaj gulf xxmaj war we re back at it but with xxup isis this time.the xxup u.s . xxmaj air xxmaj force has deployed xxup b-52 long - ranger bombers to the xxmaj middle xxmaj east , for the first time since the xxmaj gulf xxmaj war ended , to conduct strikes against the xxmaj islamic xxmaj state in xxmaj iraq and","xxmaj here we go again 25 years after the xxmaj gulf xxmaj war we re back at it but with xxup isis this time.the xxup u.s . xxmaj air xxmaj force has deployed xxup b-52 long - ranger bombers to the xxmaj middle xxmaj east , for the first time since the xxmaj gulf xxmaj war ended , to conduct strikes against the xxmaj islamic xxmaj state in xxmaj iraq and xxmaj"
1,reuters on xxmaj friday . i m taking my chances . xxmaj we can not live in fear and let the fear paralyze us . xxmaj she said the party was still in contact with those members outside the country as the opposition tries to rally international pressure on the government . a spokesman for the ruling party said it was an internal issue for the xxup cnrp if its members were,on xxmaj friday . i m taking my chances . xxmaj we can not live in fear and let the fear paralyze us . xxmaj she said the party was still in contact with those members outside the country as the opposition tries to rally international pressure on the government . a spokesman for the ruling party said it was an internal issue for the xxup cnrp if its members were abroad
2,"29 , took the whole family his little four children and wife to the is - controlled area inside xxmaj syria . xxmaj now xxmaj skr mo filmed a propaganda video outside the xxmaj syrian city xxmaj kobane where he preaches jihad and calls xxmaj swedish jihadists to leave xxmaj sweden and join the holy war . xxmaj my brothers , hijra ( migration ) and jihad are so simple . xxmaj",", took the whole family his little four children and wife to the is - controlled area inside xxmaj syria . xxmaj now xxmaj skr mo filmed a propaganda video outside the xxmaj syrian city xxmaj kobane where he preaches jihad and calls xxmaj swedish jihadists to leave xxmaj sweden and join the holy war . xxmaj my brothers , hijra ( migration ) and jihad are so simple . xxmaj it"
3,"xxmaj committee in xxmaj july . xxmaj american xxunk s xxmaj hillary lying about a video being reason for the attack on the unsecured xxmaj benghazi compound , even though she xxup always xxup knew ( see email above ) that it had xxup nothing to do with the attack : here s the speech xxmaj sean xxmaj smith s mother , xxmaj pat xxmaj smith gave at the xxup rnc xxmaj","committee in xxmaj july . xxmaj american xxunk s xxmaj hillary lying about a video being reason for the attack on the unsecured xxmaj benghazi compound , even though she xxup always xxup knew ( see email above ) that it had xxup nothing to do with the attack : here s the speech xxmaj sean xxmaj smith s mother , xxmaj pat xxmaj smith gave at the xxup rnc xxmaj convention"
4,"while these situations are often called accidents , the true issue is negligence . xxmaj simply put , nothing would have happened in many , if not most accidental shootings if the gun wasn t present in the first place or , at minimum , if the owners of the weapons actually cared about gun xxunk xxup nra regularly promotes the lie that all people who own firearms are responsible good guys","these situations are often called accidents , the true issue is negligence . xxmaj simply put , nothing would have happened in many , if not most accidental shootings if the gun wasn t present in the first place or , at minimum , if the owners of the weapons actually cared about gun xxunk xxup nra regularly promotes the lie that all people who own firearms are responsible good guys with"


Fine tuning el modelo de lenguaje.

In [21]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy,Perplexity()]).to_fp16()

In [22]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.630003,3.489906,0.355305,32.782867,17:59


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3) 

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.308049,3.334590,0.374641,28.066879,19:23
1,3.281151,3.297311,0.379551,27.039839,19:36


In [ ]:
learn.save_encoder('finetuned')

Creando un clasificador.

In [ ]:
dls_clas = TextDataLoaders.from_csv('dataSetConcatenated/', 'concatenated.csv', text_col='title', label_col='class')

In [ ]:
dls_clas.show_batch(max_n=5)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))